In [1]:
import torch
import clip
import laion_clap

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.is_available())

True


In [14]:
import clip
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
#device = torch.device("cuda:0")
model1, preprocess1 = clip.load("ViT-B/32", device=device)

In [15]:
import pandas as pd
print(pd.show_versions)

<function show_versions at 0x7f27ce00a3b0>


In [16]:
def extract_text_from_csv(csv_path, column_name):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)
        
        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()
        print(len(text_data_list))
        i = 1
        # テキストデータを順に表示
        for text_data in text_data_list[0:20]:
            print("num:",i)
            print(text_data)
            try:
                text_embed = model1.encode_text(clip.tokenize(text_data).to(device))
                #text_embed = clip.tokenize(text_data).to(device)
                print(text_embed.shape)
            
            except Exception as e:
                print(f"エラーが発生しました: {e}")
            
            i = i+1

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# 以下は使用例です
csv_file_path = "../Datasets/archive/musiccaps-public.csv"
target_column_name = "aspect_list"

extract_text_from_csv(csv_file_path, target_column_name)


5521
num: 1
['low quality', 'sustained strings melody', 'soft female vocal', 'mellow piano melody', 'sad', 'soulful', 'ballad']
torch.Size([1, 512])
num: 2
['guitar song', 'piano backing', 'simple percussion', 'relaxing melody', 'slow tempo', 'bass', 'country feel', 'instrumental', 'no voice']
torch.Size([1, 512])
num: 3
['amateur recording', 'finger snipping', 'male mid range voice singing', 'reverb']
torch.Size([1, 512])
num: 4
['backing track', 'jazzy', 'digital drums', 'piano', 'e-bass', 'trumpet', 'acoustic guitar', 'digital keyboard song', 'medium tempo']
torch.Size([1, 512])
num: 5
['rubab instrument', 'repetitive melody on different octaves', 'no other instruments', 'plucked string instrument', 'no voice', 'instrumental', 'fast tempo']
torch.Size([1, 512])
num: 6
['instrumental', 'white noise', 'female vocalisation', 'three unrelated tracks', 'electric guitar harmony', 'bass guitar', 'keyboard harmony', 'female lead vocalisation', 'keyboard harmony', 'slick drumming', 'boomy ba

In [12]:
import pandas as pd

def process_text_and_store(csv_path, column_name):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)
        
        # 新しい列 "clip_text_embed" を追加
        df["clip_text_embed"] = ""

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()
        # テキストデータを順に処理し、"clip_text_embed" 列に格納
        for i, text_data in enumerate(text_data_list[0:500]):
            try:
                # ここにテキストデータを処理するコードを追加（例：何かしらの変換）
                tokened = clip.tokenize(text_data).to(device)
                processed_text = model1.encode_text(tokened)

                # "clip_text_embed" 列に格納
                df.at[i, "clip_text_embed"] = processed_text
                del tokened
                del processed_text
                torch.cuda.empty_cache()
                
            except Exception as e:
                # エラーが発生した場合、エラーメッセージを格納
                df.at[i, "clip_text_embed"] = "ERROR"

        # 新しいデータフレームを表示
        print(df)

        # 新しいCSVファイルとして保存（任意）
        df.to_csv("../Datasets/archive/musiccaps-processed.csv", index=False)
    except Exception as e:
        print(f"エラーが発生しました: {e}")

# 以下は使用例です
csv_file_path = "../Datasets/archive/musiccaps-public.csv"
target_column_name = "aspect_list"

process_text_and_store(csv_file_path, target_column_name)

             ytid  start_s  end_s  \
0     -0Gj8-vB1q4       30     40   
1     -0SdAVK79lg       30     40   
2     -0vPFx-wRRI       30     40   
3     -0xzrMun0Rs       30     40   
4     -1LrH01Ei1w       30     40   
...           ...      ...    ...   
5516  zw5dkiklbhE       15     25   
5517  zwfo7wnXdjs       30     40   
5518  zx_vcwOsDO4       50     60   
5519  zyXa2tdBTGc       30     40   
5520  zzNdwF40ID8       70     80   

                               audioset_positive_labels  \
0                          /m/0140xf,/m/02cjck,/m/04rlf   
1     /m/0155w,/m/01lyv,/m/0342h,/m/042v_gx,/m/04rlf...   
2                                   /m/025_jnm,/m/04rlf   
3                                    /m/01g90h,/m/04rlf   
4                                   /m/02p0sh1,/m/04rlf   
...                                                 ...   
5516                                /m/01sm1g,/m/0l14md   
5517                      /m/02p0sh1,/m/04rlf,/m/06j64v   
5518  /m/01glhc,/m/02sgy

In [16]:
import pandas as pd

def process_and_add_embedding(csv_path, column_name):
    try:
        # CSVファイルの読み込み
        df = pd.read_csv(csv_path)

        # 新しい列 "clip_text_embed" を追加
        df["clip_text_embed"] = None

        # 指定された列のデータをリストとして取得
        text_data_list = df[column_name].tolist()

        # テキストデータを順に処理して "clip_text_embed" に格納
        for i, text_data in enumerate(text_data_list[0:500]):
            try:
                # ここでテキストデータを512次元のテンソルに変換する処理を実装
                # 以下は仮の例としてランダムな512次元ベクトルにしています
                token = clip.tokenize(text_data).to(device)
                embedding_tensor = model1.encode_text(token)
                
                # "clip_text_embed" 列にテンソルを格納
                df.at[i, "clip_text_embed"] = embedding_tensor
                
                del token
                del embedding_tensor
                torch.cuda.empty_cache()
                
            except Exception as e:
                # エラーが発生した場合、例外メッセージを格納
                df.at[i, "clip_text_embed"] = "ERROR"
                print(str(e))

        # 結果を表示
        print(df)

        # CSVファイルに結果を保存
        df.to_csv("../Datasets/archive/musiccaps-processed.csv", index=False)

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# 以下は使用例です
csv_file_path = "../Datasets/archive/musiccaps-public.csv"
target_column_name = "aspect_list"

process_and_add_embedding(csv_file_path, target_column_name)

Input ['live performance', 'poor audio quality', 'ambient noises', 'school dance program', 'crowd noise', 'high gain volume', 'hissing tones', 'medium tempo', 'techno dance song', 'male singer', 'groovy bass line', 'techno dance rhythm', 'keyboard accompaniment', 'kids dancing', 'noisy crowds', 'home video', 'school program', 'noise'] is too long for context length 77
Input ['guitarist', 'male talking', 'twang sounds', 'mediocre audio quality', 'guitar string tones', 'guitar technique', 'fretboard sounds', 'guitar demo', 'online guitar lessons', 'home music video', 'youtube tutorial', 'bas]d audio quality', 'humming noise', 'energetic', 'passionate', 'spirited okay', 'strong voice', 'ambient room noises'] is too long for context length 77
Input ['male vocalist', 'medium fast tempo', 'monologue', 'background music', 'hit songs', 'retro', 'narrator', 'narration', 'documentary', 'animated singing', 'enthusiastic drumming', 'cymbal ride', 'electric guitar', 'groovy bass lines', 'rock and r